## Whole Foods Helper:

In [1]:
DO_LOGIN_AND_NAVIGATE=False
DO_DELIVERY_SCAN=True

In [2]:

import sel_scrape as sc
import pandas as pd
import sys,os
import datetime
import urllib.request as ur
import importlib
import datetime
import time
import pdb
import pathlib
MY_HOME = str(pathlib.Path.home())

In [3]:
pd.set_option('display.max_colwidth', -1)
PROFILE_PATH = f'{MY_HOME}/Library/Application Support/Google/Chrome/Default'

/Users/bperlman1/Virtualenvs3/dashrisk4/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


### Try to login and navigate amazon whole foods
This does not yet work b/c amazon senses automation

In [4]:
def __loop(slp_tm):
    for _ in range(100):
        try:
            print('start')
            sac = sc.SelScrape(headless=False,executable_path='./drivers/geckodriver')

            ff = open('./temp_folder/smh.txt','r').read()
            ffs = ff.split(',')
            u = ffs[0]
            p = ffs[1]
            w = ffs[2]
            sac.goto(w)

            xp = '//input[@id="ap_email"]'
            sac.sendkeys(xp,u)
            xp2 = '//input[@id="continue"]'
            result = sac.click_element(xp2)
            if result['status'] is not None:
                raise ValueError(result['status'])
            sac.wait_implicitly(10)
            time.sleep(slp_tm)

            xp = '//input[@id="ap_password"]'
            sac.sendkeys(xp,p)
            xp2 = '//input[@id="signInSubmit"]'
            result = sac.click_element(xp2)
            if result['status'] is not None:
                raise ValueError(result['status'])
            sac.wait_implicitly(10)

            xp2 = '//a[@id="nav-cart"]'
            result = sac.click_element(xp2)
            if result['status'] is not None:
                raise ValueError(result['status'])
            sac.wait_implicitly(10)
            time.sleep(slp_tm)
            
            xp2 = "//input[contains(@name,'proceedToALMCheckout')]"
            result = sac.click_element(xp2)
            if result['status'] is not None:
                raise ValueError(result['status'])
            sac.wait_implicitly(10)
            time.sleep(slp_tm)

            xp2 = "//span[@id='a-autoid-0']//a[@name='proceedToCheckout']"
            result = sac.click_element(xp2)
            if result['status'] is not None:
                raise ValueError(result['status'])
            sac.wait_implicitly(10)
            time.sleep(slp_tm)

            xp2 = "//span[@id='subsContinueButton']//input[@class='a-button-input']"
            result = sac.click_element(xp2)
            if result['status'] is not None:
                raise ValueError(result['status'])
            sac.wait_implicitly(10)
            time.sleep(slp_tm)
            
            time.sleep(10)
            r = find_available(sac)
            if r:
                print("success")
            else:
                print("failure")
            
        except Exception as e:
            print(str(e))
        sac.driver.quit()
        time.sleep(60*60)


In [5]:
if DO_LOGIN_AND_NAVIGATE:
    __loop()

### Only refresh and scrape an already logined in amazon whole foods page
This works, since you are only refreshing a page, and not clicking it

In [6]:
def find_available(sac):
    t = datetime.datetime.now()
    tm = t + datetime.timedelta(1)
    t_yyyymmdd = str(int(t.year)*100*100 + int(t.month)*100 + int(t.day))
    tm_yyyymmdd = str(int(tm.year)*100*100 + int(tm.month)*100 + int(tm.day))
    but1 = f"//button[@name='{t_yyyymmdd}']//div[@class='ufss-date-select-toggle-text-availability']"
    but1_elem = sac.findxpath(but1)
    if but1_elem['status'] is None:
        but1_elem = but1_elem['value'][0].text
    print(but1_elem)

    but2 = f"//button[@name='{tm_yyyymmdd}']//div[@class='ufss-date-select-toggle-text-availability']"
    but2_elem = sac.findxpath(but1)
    if but2_elem['status'] is None:
        but2_elem = but2_elem['value'][0].text
    else:
        but2_elem = 'not'
    print(but2_elem)
    cond1 = 'not' in but1_elem.lower()
    cond2 = 'not' in but2_elem.lower()
    if not (cond1 and cond2):
        return True
    return False

### create a webdriver

In [7]:
if DO_DELIVERY_SCAN:    
    pp=PROFILE_PATH
    sac = sc.SelScrape(headless=False,driver_name='chrome',
                      profile_path=pp)
    sac.wait_implicitly(2)


## You must now login to amazon/whole foods, add stuff to a shopping cart, and navigate through checkout until you get to the page that shows you available delivery dates

#### The cell below pauses the workbook cell execution so that you can login, and navigate to the page above
Hit enter to continue to running the loop that checks for time slots

In [8]:
input("Hit enter to continue AFTER YOU HAVE LOGGED IN AND NAVIGATED THROUGH CHECKOUT: ")

Hit enter to continue AFTER YOU HAVE LOGGED IN AND NAVIGATED THROUGH CHECKOUT: 


''

### Execute loop that checks to see if a time slot is available on whole foods

#### When a refresh of the delivery date page shows an available day, the loop will:
1. Start to beep
2. Execute an input statement.  This is to allow you to manually select the available date, and finish checkout.
3. **!!!!!!!!!! DON'T HIT ENTER UNTIL YOU HAVE COMPLETED CHECKOUT !!!!!!!!!!**
4. When you have finished checkout you can hit enter, and then hit the stop button in Jupyter Notebook.

In [9]:
# every minute
# execution_minute_list = list(range(0,60))
# only important minutes for wf
execution_minute_list = [0,1] + list(range(11,52,10)) + [58,59]


In [10]:
def check_wfoods():
    url = sac.curr_url()
    sac.goto(url)
    time.sleep(1)
    t = True
    try:
        t = find_available(sac)
    except:
        pass
    if t:
        print(f"{datetime.datetime.now()}")
        for _ in range(30):
            os.system("echo -ne '\007'")
            time.sleep(.5)
        input('waiting to attempt to get delivery: hit return when finished')
    

In [11]:
import datetime
import time

def wait_for_00(m,r=1000):
    for _ in range(r):
        n = datetime.datetime.now()
        nt = n.time()
        s = 60-nt.second
        time.sleep(s)
        nt = datetime.datetime.now().time()
        if nt.minute in (m):
            check_wfoods()
            return


In [12]:
if DO_DELIVERY_SCAN:        
    r = execution_minute_list
    print(",".join([str(x) for x in r]))
    for _ in range(60*10):
        wait_for_00(r)

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
{'status': 'not found', 'value': None}
not
2020-04-18 18:36:06.286198


KeyboardInterrupt: 

In [ ]:
# wait_time_between_scans = 60*5
# if DO_DELIVERY_SCAN:        
#     for i in range(60*10):
#         url = sac.curr_url()
#         sac.goto(url)
#         t = True
#         try:
#             t = find_available(sac)
#         except:
#             pass
#         if t:
#             print(f"{datetime.datetime.now()}")
#             for _ in range(30):
#                 os.system("echo -ne '\007'")
#                 time.sleep(.5)
#             input('waiting to attempt to get delivery: hit return when finished')
#         time.sleep(wait_time_between_scans)